In [1]:
import pandas as pd
import numpy as np


In [2]:
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())

In [4]:
#Step2 Split workload_A by creating different dataframes

workload_A_ids = workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = workload_A[workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1

In [5]:
#Step3 Get metrics

tmpframe = workload_A.T.copy()
tmpframe = tmpframe.tail(585-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

In [9]:
#Step4 FA (apply for all workloads)
from sklearn.decomposition import FactorAnalysis


transformer = FactorAnalysis(n_components=7, random_state=0)
workload_list_afterFA = []

for i in range(num_workloads_A):
    tmp_workload_A_listformetrics = workload_A_listformetrics[i]
    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values
    tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    tmp_transformer = FactorAnalysis(n_components=7, random_state=0)
    tmp_workload_A_listformetrics_array_Trans_transformed = transformer.fit_transform(tmp_workload_A_listformetrics_array_Trans)
    workload_list_afterFA.append(tmp_workload_A_listformetrics_array_Trans_transformed)

In [10]:
#Step5 K-means (apply for all workloads)
from sklearn.cluster import KMeans


workload_list_centers_metrics_name = []

for i in range(num_workloads_A):
    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=2, random_state=0).fit(workload_list_afterFA[i])
    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_
    tmp_centers_index = np.where((workload_list_afterFA[i]==tmp_cluster_centers[:,None]).all(-1))
    tmp_centers_metrics_name = []
    tmp_n_centers = len(tmp_centers_index)
    for j in range(tmp_n_centers):
        tmp_mname = metrics_name[tmp_centers_index[j][0]]
        tmp_centers_metrics_name.append(tmp_mname)
    
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)

In [13]:
#Step6 Get Prutiend workload (apply for all workloads)

knobs_latency_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']
latency = ['latency']

pruning_workload_list = []

for i in range(num_workloads_A):
    tmp_pruning_list = knobs_latency_list + latency + workload_list_centers_metrics_name[i]
    tmp_pruning_workload_A = (workload_A_list[i])[tmp_pruning_list]
    pruning_workload_list.append(tmp_pruning_workload_A)

In [21]:
#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

workload_A_GPRmodels_lists = []

for i in range(num_workloads_A):
    tmp_GPRmodels_lists = []
    
    tmp_Pruning_workload_A_X = ((workload_A_list[i])[knobs_latency_list]).values
    tmp_Pruning_workload_A_y_o = ((workload_A_list[i])[latency]).values
    tmp_kernel = DotProduct() + WhiteKernel()
    tmp_workload_A_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Pruning_workload_A_X, tmp_Pruning_workload_A_y_o)
    
    tmp_GPRmodels_lists.append(tmp_workload_A_gpr_o)
    
    tmp_num_centers = len(workload_list_centers_metrics_name[i])
    for j in range(tmp_num_centers):
        tmp_Pruning_workload_A_y_m = ((workload_A_list[i])[workload_list_centers_metrics_name[i][j]]).values
        tmp_kernel = DotProduct() + WhiteKernel()
        tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Pruning_workload_A_X, tmp_Pruning_workload_A_y_m)
        tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
    
    workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)

G:\Users\28215\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:469: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00013058, -0.00059283]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 61, 'nit': 12, 'warnflag': 2}
  ConvergenceWarning)
G:\Users\28215\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:469: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.84483485e-04,  7.72853886e-06]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 7, 'warnflag': 2}
  ConvergenceWarning)


In [23]:
#*************************************************************************************************************************************************
#    1
#    We have workload_A_list contain 58 workloads
#    workload_A_list
#
#    2
#    We have workload_list_centers_metrics_name contain 58 lists, each list have the metrics kept after FA and K-means for each workload
#    workload_list_centers_metrics_name
#
#    3
#    We have pruning_workload_list contain 58 workloads after prunning
#    pruning_workload_list
#
#    4
#    We have workload_A_GPRmodels_lists contain 58 lists, each list have 1(latency) + k(number of metrics kept) GPR models
#    workload_A_GPRmodels_lists
#*************************************************************************************************************************************************

In [24]:
#Phase2 Using the online workload B as the validation data

#In workloads B, each workload has 6 data points, we use 5 of them apply workload mapping, and use the left 1 as the target workload, predict
#latency and compare it with the actual value in workloads B.

In [25]:
#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

In [26]:
#Step2 Split workload_B by creating different dataframes

workload_B_ids = workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = workload_B[workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

In [27]:
#Step3 Split each workload into 5 data points and 1 data point as target

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])

In [29]:
#Step4 workload mapping Test for all workload in workload B
all_distance_list = []
for i in range(100):
    tmp_test_workload_B_mapping = workload_B_mapping_data[i]
    tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_latency_list]).values
    tmp_distance_list = []
    
    for j in range(num_workloads_A):
        tmp_test_workload_A = workload_A_list[j]
        tmp_test_workload_A_metrics = workload_list_centers_metrics_name[j]
        tmp_test_workload_A_gpr_model = workload_A_GPRmodels_lists[j]
        tmp_test_prunning_workload_A = pruning_workload_list[j]
    
        tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_latency_list]).values
    
        tmp_corr_metric_values = []
        
        for k in range(5):
            tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
            tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
            if len(tmp_check[0]) == 0:
                tmp_gpr_predic = []
                for m in range(2):
                    tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[m+1].predict([tmp_test_workload_B_mapping_cfigs[m]])[0])
                tmp_corr_metric_values.append(tmp_gpr_predic)
            
            else:
                tmp_gpr_predic = []
                tmp_row_idx = tmp_check[0][0]
                #print(tmp_row_idx)
                #print('xxxxxxxxxxxxxxxxxxx')
                tmp_test_prunning_workload_A_array = tmp_test_prunning_workload_A.values
                tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13])
                tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][14])
                tmp_corr_metric_values.append(tmp_gpr_predic)
        
        tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[tmp_test_workload_A_metrics].values
        #print(tmp_test_workload_B_mapping_metrics_value)
        #print('--------')
        tmp_distance = np.sum((tmp_corr_metric_values-tmp_test_workload_B_mapping_metrics_value)*(tmp_corr_metric_values-tmp_test_workload_B_mapping_metrics_value))
        tmp_distance_list.append(tmp_distance)
     
    all_distance_list.append(tmp_distance_list)


    
all_min_idx_list = []
for i in range(100):
    tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
    all_min_idx_list.append(tmp_idx)

In [30]:
#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

predictGPRmodel_list = []

for i in range(100):
    tmp_similar_idx = all_min_idx_list[i]
    tmp_similar_workload = pruning_workload_list[tmp_similar_idx]
    tmp_similar_workload_configs = ((pruning_workload_list[tmp_similar_idx])[knobs_latency_list]).values
    tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    
    tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
    tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
    tmp_workload_B_configs = workload_B_mapping_data[i][knobs_latency_list].values
    #print(tmp_workload_B_configs)
    
    for j in range(5):
        tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
        tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_mapping_metrics_with_similar[j]
        tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
        if len(tmp_check[0]) == 0:
            tmp_Augmented_pruned_target_configs = np.insert(tmp_Augmented_pruned_target_configs,0,tmp_test_workload_B_mapping_cfigs,axis=0)
            tmp_Augmented_pruned_target_metrics = np.insert(tmp_Augmented_pruned_target_metrics,0,tmp_test_workload_B_mapping_metrics_with_similar,axis=0)
        else:
        
            tmp_row_idx = tmp_check[0][0]
            #print(tmp_row_idx)
            #print('xxxxxxxxxxxxxxxxxxx')
            tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
    test_workload_B_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Augmented_pruned_target_configs, tmp_Augmented_pruned_target_metrics.T[0])
    predictGPRmodel_list.append(test_workload_B_gpr_o)

G:\Users\28215\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:469: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([6.91159668e-05, 7.95012220e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 66, 'nit': 16, 'warnflag': 2}
  ConvergenceWarning)


In [31]:
#Step6 predict for all workload B

for i in range(5):
    tmp_target_config = workload_B_target[i][knobs_latency_list].values
    tmp_predict = predictGPRmodel_list[i].predict(tmp_target_config)
    print(tmp_predict)
    print(workload_B_target[i][latency])
    print('------------')

[6.01275399]
   latency
5    7.923
------------
[468.89355157]
      latency
11  260.62625
------------
[140.57925878]
     latency
17  109.0855
------------
[423.68501883]
    latency
23   61.997
------------
[190.90305545]
     latency
29  89.78475
------------
